In [1]:
import img2pdf
import re
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imutils
import cv2
import os
import io
from google.cloud import vision
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"E:/ssl_invoice/mapping-customers.json"

In [3]:
#im_dir = "E:/ssl_invoice/aligned_samples"
#im_dir = "E:/ssl_invoice/aligned_images"
im_dir = "E:/ssl_invoice/aligned_images/local_4.jpg"
#list_of_images_paths = [os.path.join(im_dir, file) for file in os.listdir(im_dir) 
                       #if file.endswith(".jpg") or file.endswith(".png")]
#invoice_images = [cv2.imread(img_path) for img_path in list_of_images_paths]
invoice_images = cv2.imread(im_dir)
#print(invoice_images)
#plt.imshow(invoice_images)
#print(len(invoice_images))

In [4]:
def merge_texts(symbols):
    text = ""
    for symbol in symbols:
        text += symbol.text
    return text 

def get_document_bounds(image_file):
    """
    Returns document bounds given an image.
    """
    client = vision.ImageAnnotatorClient()

    bounds = []
    texts = []

    with io.open(image_file, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image, 
                                              image_context={"language_hints": ["bn"]})
    document = response.full_text_annotation

    # Collect word level bounds by enumerating all document features
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    bounds.append(word.bounding_box)
                    texts.append(merge_texts(word.symbols))


    # The list `bounds` contains the coordinates of the bounding boxes.
    # The list `texts` contains the corresponding texts
    return bounds, texts

In [5]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"E:/ssl_invoice/mapping-customers.json"

In [6]:
doc_bounds = []
doc_texts = []
#for n,image_file in enumerate(list_of_images_paths):
    #print(image_file)
doc_bounds,doc_texts = get_document_bounds(im_dir)
    #doc_bounds.append(bounds)
    #print(n)
    #doc_texts.append(texts)
#doc_bounds,doc_texts = get_document_bounds(im_dir)

In [7]:
print(doc_texts)

['Software', 'Shop', 'Limited', 'Head', 'Office', ':', '93', 'B', ',', 'New', 'Eskaton', 'Road', ',', 'Dhaka', '-', '1000', ',', 'Bangladesh', '.', 'Branch', 'Office', ':', 'Standard', 'Center', ',', 'Level', '4', ',', '5', ',', '7', '&', '9', ',', '27/1', ',', 'New', 'Eskaton', 'Road', ',', 'Dhaka', '-', '1000', 'Chattogram', 'Office', ':', '318', ',', 'B.M', 'Height', ',', 'Level', '2', ',', 'Sheikh', 'Mujib', 'Road', ',', 'Agrabad', ',', 'Badamtoli', ',', 'Chattogram', 'Tel', ':', '+880', '961', '222', '1000', ',', '+880', '2', '4831', '6009', ',', '+880', '24831', '5056', ',', 'Email', ':', 'info@sslwireless.com', ',', 'Website', ':', 'www.sslwireless.com', 'G', '©', 'SSL', 'WIRELESS', 'LankaBangla', 'Finance', 'Ltd', 'Safura', 'Tower', '(', 'Level', ':', '11', ')', '20', ',', 'Kemal', 'Ataturk', 'Avenue', 'Banani', ',', 'Dhaka', '-', '1213', ',', 'Bangladesh', 'Attn', ':', 'Muhammad', 'Habib', 'Haider', 'Head', 'of', 'GIS', 'INVOICE', 'No.', 'SSL', '/', '21', '/', '041127', 'Day',

In [8]:
 #doc_texts[0]

In [9]:
#let's parse a shop name from invoice samples
shop_name=[]
#for text in doc_texts:
#if first word is invoice let's take 2nd document texts
if doc_texts[0]=="Invoice":
    shop_name.append(doc_texts[1])
#else take 1st three document texts
else:
    shop_name.append(doc_texts[0]+" "+doc_texts[1]+" "+doc_texts[2])
    
print("Parsing Shop Name:",shop_name)     

Parsing Shop Name: ['Software Shop Limited']


In [10]:
#for multiple invoice samples reading a path
im_dir = "E:/ssl_invoice/aligned_samples"
list_of_images_paths = [os.path.join(im_dir, file) for file in os.listdir(im_dir) 
                       if file.endswith(".jpg") or file.endswith(".png")]
#invoice_images = [cv2.imread(img_path) for img_path in list_of_images_paths]

In [11]:

#let's parse a shop name from multiple invoice samples
for n,image_file in enumerate(list_of_images_paths):
    #print(image_file)
    doc_bounds,doc_texts = get_document_bounds(image_file)
    #if first word is invoice let's take 2nd document texts
    if doc_texts[0]=="Invoice":
        shop_name=doc_texts[1]
        print("Filename: {} | shop_name:{}".format(image_file.split("/")[-1],shop_name))
    #else take 1st three document texts
    else:
        shop_name=doc_texts[0]+" "+doc_texts[1]+" "+doc_texts[2]
        print("Filename: {} | shop_name:{}".format(image_file.split("/")[-1],shop_name))
    

Filename: aligned_samples\local_1.jpg | shop_name:RoyalFruitz.com
Filename: aligned_samples\local_10.jpg | shop_name:Software Shop Limited
Filename: aligned_samples\local_11.jpg | shop_name:Software Shop Limited
Filename: aligned_samples\local_12.jpg | shop_name:Software Shop Limited
Filename: aligned_samples\local_13.jpg | shop_name:edhe Head Office
Filename: aligned_samples\local_14.jpg | shop_name:Software Shop Limited
Filename: aligned_samples\local_15.jpg | shop_name:Software Shop Limited
Filename: aligned_samples\local_2.jpg | shop_name:Software Shop Limited
Filename: aligned_samples\local_3.jpg | shop_name:Software Shop Limited
Filename: aligned_samples\local_4.jpg | shop_name:Software Shop Limited
Filename: aligned_samples\local_5.jpg | shop_name:Software Shop Limited
Filename: aligned_samples\local_6.jpg | shop_name:Software Shop Limited
Filename: aligned_samples\local_8.jpg | shop_name:Software Shop Limited
Filename: aligned_samples\local_9.jpg | shop_name:Software Shop Limit

In [12]:
 
    #doc_bounds.append(bounds)

In [13]:
#x=(top_left_x+top_right_x)
#doc_dfs[0].top_left_x+doc_dfs[0].top_right_x

In [14]:
#doc_dfs[0]["word"].iloc[1]

In [15]:
#len(doc_dfs[0]["word"])

In [16]:
#l=round(len(doc_dfs[0]["word"])/7)

In [17]:
#doc_dfs[0]["word"][:l]

In [18]:
#find biggest text from  top_corner

#all_text=[]
#for text in doc_dfs[0]["word"][:l]:
    #all_text.append(text)   
#print(all_text)
#max_t=max(all_text)
#print(max_t)

In [19]:
#doc_dfs